In [1]:
# Import
import numpy as np
import matplotlib.pyplot as plt
import datetime as DT
import pandas as pd
import math
import scipy
from IPython.core.debugger import Tracer
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.classification import log_loss

from sklearn import datasets, linear_model
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.svm import SVC, SVR
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

%matplotlib inline
import seaborn as sns; sns.set()
import operator
import itertools
import matplotlib.patches as mpatches
from scipy.stats.stats import pearsonr

import warnings
warnings.filterwarnings('ignore')
#Tracer()()
def prep (a):
    return a.reshape(a.shape[0], 1)
def unprep (a):
    return np.squeeze(a)
def figurefullwidth():
    plt.figure(figsize=(25,5));
def subplot(a):
    plt.subplot(1,3,a);
def subplottitle(a, b):
    ax = plt.subplot(1, 3, a)
    ax.set_title(b)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [4]:
def encode_onehot(df, cols):
    """
    One-hot encoding is applied to columns specified in a pandas DataFrame.
    
    Modified from: https://gist.github.com/kljensen/5452382
    
    Details:
    
    http://en.wikipedia.org/wiki/One-hot
    http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(outtype='records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = df.join(vec_data)
    return df



In [11]:
def processfile(csv):
    #load data
    df = pd.read_csv(csv, header= 0)
    del df['order_ref']
    df["inplay"] = df["inplay"].astype(int)
    df["strategy"] = df["strategy"].apply(lambda x: 0 if x == "A" else 1)
    #df["pct_matched"].unique()
    df["pct_matched"].isnull().sum()

    #list of index with nan
    nans = df['pct_matched'].index[df['pct_matched'].apply(np.isnan)]
    #df.loc[3173]
    df = df.drop(df.index[nans])
    r = df['bet_type'].apply(lambda x: pd.Series(x.split('|')))
    del r[0]
    for i in range(0, r.shape[1]):
        df["bet_type" + str(i)] = r[i+1]
    del df["bet_type"]
    df["pct_matched_cat"] = df["pct_matched"].apply(lambda x: 1 if x > 0.5 else -1)
    
    df = encode_onehot(df, ["bookie", "bet_type0"])
    del df["bet_type1"]
    del df["bet_type2"]
    del df["bet_type3"]
    del df["pct_matched"]
    
    #dfy = pd.concat(df["pct_matched_cat"])
    #dfy = DataFrame()
    dfy = df["pct_matched_cat"]
    del df["pct_matched_cat"]
    #df_y["pct_matched_cat"]
    return df, dfy

In [12]:
dftrain_x, dftrain_y = processfile('/Users/david/notebooks/data/training_set.csv')
dftrain_x.head()
dftrain_x.columns
dftrain_y.head()
#dftrain_y.columns

,strategy,inplay,age,stakes,max_stakes,price,pct_max_stake_reqd_overall,fallback_margin,arb_margin,fallback_cost,...,bet_type0=team_total_goals,bet_type0=team_win_to_nil,bet_type0=total_goals,bet_type0=total_goals_2,bookie=A,bookie=B,bookie=C,bookie=D,bookie=E,bookie=F
0,0,1,896.968049,2.429000,2.429,2.428571,100.000000,0.058410,0.403999,0.345589,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,0,0.432858,2.380916,751.490,9.200000,0.316826,-0.176400,0.064876,0.241276,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,1,0.064819,13.634291,2338.670,1.240000,0.582993,-0.292514,0.363597,0.656111,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,0,107.238326,18.846154,19.820,2.600000,95.086548,0.041334,0.495772,0.454439,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,1,38.344546,40.020000,40.020,6.000000,100.000000,0.200951,0.485378,0.284427,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Index(['strategy', 'inplay', 'age', 'stakes', 'max_stakes', 'price',
       'pct_max_stake_reqd_overall', 'fallback_margin', 'arb_margin',
       'fallback_cost', 'bet_type0=asian_handicap',
       'bet_type0=both_teams_to_score', 'bet_type0=clean_sheet',
       'bet_type0=correct_score', 'bet_type0=correct_score_other',
       'bet_type0=double_chance', 'bet_type0=draw_no_bet',
       'bet_type0=euro_handicap', 'bet_type0=match_odds',
       'bet_type0=match_odds_and_btts', 'bet_type0=match_odds_and_ou',
       'bet_type0=odd_or_even', 'bet_type0=over_under',
       'bet_type0=team_over_under', 'bet_type0=team_total_goals',
       'bet_type0=team_win_to_nil', 'bet_type0=total_goals',
       'bet_type0=total_goals_2', 'bookie=A', 'bookie=B', 'bookie=C',
       'bookie=D', 'bookie=E', 'bookie=F'],
      dtype='object')

0   -1
1    1
2   -1
3    1
4   -1
Name: pct_matched_cat, dtype: int64

AttributeError: 'Series' object has no attribute 'columns'

In [13]:
dftest_x, dftest_y = processfile('/Users/david/notebooks/data/testing_set.csv')
dftest_x.head()
dftest_x.columns
dftest_y.head()
#dftest_y.columns

,strategy,inplay,age,stakes,max_stakes,price,pct_max_stake_reqd_overall,fallback_margin,arb_margin,fallback_cost,...,bet_type0=team_total_goals,bet_type0=team_win_to_nil,bet_type0=total_goals,bet_type0=total_goals_2,bookie=A,bookie=B,bookie=C,bookie=D,bookie=E,bookie=F
0,0,1,61.637285,4.330000,4.3300,2.320000,100.000000,0.410490,0.794893,0.384403,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,0,72.134169,28.839624,76.8258,2.219512,37.538982,-0.181804,0.012743,0.194547,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,1,0.298152,8.228528,5062.0000,1.760000,0.162555,-0.408819,0.010379,0.419198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1,1,0.606424,3.123076,9.8700,8.200000,31.642105,-0.185425,0.119250,0.304676,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1,1,21.131292,25.121046,4050.0000,1.720000,0.620273,0.092797,0.194175,0.101378,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


Index(['strategy', 'inplay', 'age', 'stakes', 'max_stakes', 'price',
       'pct_max_stake_reqd_overall', 'fallback_margin', 'arb_margin',
       'fallback_cost', 'bet_type0=asian_handicap',
       'bet_type0=both_teams_to_score', 'bet_type0=clean_sheet',
       'bet_type0=correct_score', 'bet_type0=correct_score_other',
       'bet_type0=double_chance', 'bet_type0=draw_no_bet',
       'bet_type0=euro_handicap', 'bet_type0=match_odds',
       'bet_type0=match_odds_and_btts', 'bet_type0=match_odds_and_ou',
       'bet_type0=odd_or_even', 'bet_type0=over_under',
       'bet_type0=team_over_under', 'bet_type0=team_total_goals',
       'bet_type0=team_win_to_nil', 'bet_type0=total_goals',
       'bet_type0=total_goals_2', 'bookie=A', 'bookie=B', 'bookie=C',
       'bookie=D', 'bookie=E', 'bookie=F'],
      dtype='object')

0    1
1    1
2    1
3   -1
4    1
Name: pct_matched_cat, dtype: int64

Read in the Data

In [ ]:
#load data
df = pd.read_csv('/Users/david/notebooks/data/training_set.csv', header= 0)
#df["DateOfCall"] = pd.to_datetime(df["DateOfCall"]+ ' ' + df['TimeOfCall'], infer_datetime_format=True)
#a=np.datetime64('2002-06-28').astype(datetime)
del df['order_ref']
#df = df.drop(df3.columns[3], axis=1)
#df = df.drop(df3.columns[6], axis=1)
df.shape
df["inplay"] = df["inplay"].astype(int)
df["strategy"] = df["strategy"].apply(lambda x: 0 if x == "A" else 1)
df.head()


In [ ]:
#df["pct_matched"].unique()
df["pct_matched"].isnull().sum()

#list of index with nan
nans = df['pct_matched'].index[df['pct_matched'].apply(np.isnan)]
nans
#df.loc[3173]
df = df.drop(df.index[nans])

In [ ]:
df.shape
df.columns
#describe the data
df.describe()
df.describe(include = ['O'])
df.dtypes

In [ ]:
#separate the numberical and categorical clolumns
colnum = []
colcat = []
for col in df:
    if(df[col].dtype == object):
        colcat.append(col)
    else:
        colnum.append(col)
len(colnum)
len(colcat)

In [ ]:
#seaborn functions
#---------------------
#df.plot(kind="scatter", x=colnum[2], y=colnum[3])
#sns.jointplot(x=colnum[2], y=colnum[3], data=df, size=5)
#sns.FacetGrid(iris, hue="Species", size=5) \
#   .map(plt.scatter, "SepalLengthCm", "SepalWidthCm") \
#   .add_legend()
#sns.FacetGrid(iris, hue="Species", size=6) \
#   .map(sns.kdeplot, "PetalLengthCm") \
#   .add_legend()
#sns.pairplot(iris.drop("Id", axis=1), hue="Species", size=3,diag_kind="kde")

In [ ]:
#examine the numberical categories
def plotcolnumerical(c):
    a = 0
    for i in range(len(c)):
        if(a==0):
            figurefullwidth()
        subplot(a+1)
        _ = df[c[i]].hist(bins=50)
        subplottitle(a+1, c[i])
        a+=1
        if(a==3):
            a = 0
            
#plotcolnumerical(colnum)

df[colnum[2]].plot()



#df.boxplot(column='ApplicantIncome', by = 'Education')

In [ ]:
#examine the numberical categories
def plotcolnumericalagainst(c):
    a = 0
    for i in range(len(c)):
        for j in range(len(c)):
            if (a == 0):
                figurefullwidth()
            ax = plt.subplot(1, len(c), a + 1)
            x = df[c[i]]
            y = df[c[j]]
            if(x.dtype == "datetime64[ns]"):
                x = [date.to_datetime() for date in x]
            if(y.dtype == "datetime64[ns]"):
                y =  [date.to_datetime() for date in y]
                
            #subplot(a+1)
            #if(df[c[i]].dtype == datetime)
            plt.scatter(x, y, c='b')
            #_ = df[c[i]].hist(bins=50)
            ax.set_ylabel(c[j])
            ax.set_xlabel(c[i])
            #subplottitle(a+1, c[i])
            a += 1
            if (a == len(c) - 1):
                a = 0

#df.dtypes
#import matplotlib.dates as mdates
#mdates.strpdate2num('%Y-%m-%d')
#df[colnum[1]].astype(DT.datetime).values[0]
#df[colnum[1]].astype(DT.datetime).values[0].to_datetime()

#[date.to_datetime() for date in df[colnum[1]]]

#plotcolnumericalagainst(colnum[1:5])
#df.boxplot(column='ApplicantIncome', by = 'Education')

In [ ]:
#df.loc[:, df.dtypes == object].unique()
#df.loc[:, df.dtypes == object]

d = {}
d_vals = {}
for col in df:
    if(df[col].dtype == object):
        d[col] = df[col].unique()
        print(col, len(d[col]))
        d_vals[col] = df[col].value_counts()
#d
#d_vals

In [ ]:
r = df['bet_type'].apply(lambda x: pd.Series(x.split('|')))
#foo = lambda x: pd.Series([i for i in reversed(x.split('|'))])
#rev = df["bet_type"].apply(foo)
#rev


In [ ]:
r.head()

In [ ]:
del r[0]
#r.rename(columns={0:'bet_type0',1:'bet_type1',2:'bet_type2', 3:'bet_type3'},inplace=True)
r.shape
for i in range(0, r.shape[1]):
    df["bet_type" + str(i)] = r[i+1]
del df["bet_type"]
df.head()

In [ ]:
df["pct_matched_cat"] = df["pct_matched"].apply(lambda x: 1 if x > 0.5 else -1)
df.head()
df.columns

In [ ]:
#df = encode_onehot(df, ["bookie", "bet_type0", "bet_type1","bet_type2"])
#df = encode_onehot(df, ["bet_type0"])
#df = encode_onehot(df, ["bookie"])

df = encode_onehot(df, ["bookie", "bet_type0"])
df.head()
df.columns


In [ ]:
del df["bet_type1"]
del df["bet_type2"]
del df["bet_type3"]
del df["pct_matched"]
df_y["pct_matched_cat"] = df["pct_matched_cat"]
del df["pct_matched_cat"]
df.head()
df.dtypes
df_y.head()
df_y["pct_matched_cat"]

In [ ]:
df.dtypes
df.shape

In [ ]:
#look at the dist across categories
def plotcolcats(c):
    a = 0
    for i in range(len(c)):
        if(a==0):
            figurefullwidth()
        subplot(a+1)
        if(len(df[c[i]].unique())> 50):
            print ("too many cols ",len(df[c[i]].unique()), c[i], i)
        else:
            _ = df[c[i]].value_counts().plot(kind='bar')
        subplottitle(a+1, c[i])
        #plt.show()
        a+=1
        if(a==3):
            a = 0
#plotcolcats(colcat)

In [ ]:
#figurefullwidth()
#df[colcat[4]].value_counts()[:100].plot(kind='bar')
#figurefullwidth()
#df[colcat[6]].value_counts().plot(kind='bar')
#figurefullwidth()
#df[colcat[7]].value_counts()[:100].plot(kind='bar')

In [ ]:
def plotcolcatsbygroup(s, c):
    a = 0
    for i in range(len(c)):
        #if(a==0):
        #    figurefullwidth()
        #subplot(a+1)
        sizes = df.groupby([s,c[i]]).size().unstack()
        l = len(sizes)
        #print(sizes, )
        if(sizes.shape[1]> 50):
            print ("too many cols ",l, c[i], i)
        else:
            sizes.plot(kind='bar')
        #subplottitle(a+1, c[i])
        #plt.show()
        a+=1
        if(a==3):
            a = 0
#Now we start to see what is significant
#df["Year"] = df["DateOfCall"].map(lambda x: x.year)
#colcat
#df.pivot(index='Year', columns=colcat[0], values='value')
#plotcolcatsbygroup('Year', colcat)

#df.groupby(['Year',colcat[0]]).size().unstack()

#df.pivot_table(index='Year', columns=colcat[0], aggfunc=len, fill_value=0)

#plot cat counts vs numberical
'''figurefullwidth()
subplot(1)
_vc = df[colcat[0]].value_counts()
#.index.tolist()
df["Year"].values.shape
_vc.values.shape
plt.plot(df["Year"].values, _vc.values, c=_vc.index)
#df[colcat[0]].value_counts().values.tolist()
'''

In [ ]:
#look at the dist across categories
def plotcolcats(c):
    a = 0
    for i in range(10):
        if(a==0):
            figurefullwidth()
        subplot(a+1)
        if(len(df[colcat[i]].unique())> 50):
            print ("too many cols ",len(df[colcat[i]].unique()), colcat[i], i)
        else:
            _ = df[colcat[i]].value_counts().plot(kind='bar')
        subplottitle(a+1, c[i])
        #plt.show()
        a+=1
        if(a==3):
            a = 0

In [14]:
def selectcols(df1, cols, split, df2):
    t_cols = [df1.columns[i] for i in cols]
    test_l = int(df1.shape[0] * split)
    #train_l = df1.shape[0] - test_l
    print(t_cols, test_l)
    x_train = df1[t_cols][:test_l]
    x_test = df1[t_cols][test_l:]
    y_train = df2[:test_l]
    y_test = df2[test_l:]
    return x_train, x_test, y_train, y_test

In [15]:
#Start of the fitting for SVM
#------------------------------
#cols = [2, 3, 5]
#t_cols = [df.columns[i] for i in cols]
#t_cols
#df[t_cols][:345]
df.shape[1]
picks = list(range(df.shape[1]))
#picks
x_train, x_test, y_train, y_test = selectcols(dftrain_x, picks, 0.8, dftrain_y)
x_train.shape
x_test.shape
y_train.shape
y_test.shape

NameError: name 'df' is not defined

In [ ]:
def sub_p_svm (x1, y1, x2, y2, k, do_plot=False):
    #normalise the data
    scaler = StandardScaler()
    x1 = scaler.fit_transform(x1)
    x2 = scaler.transform(x2)
    
    # create feature union
    features = []
    features.append(('pca', PCA(n_components=k)))
    #features.append(('select_best', SelectKBest(k='all')))
    feature_union = FeatureUnion(features)
    # create pipeline
    estimators = []
    estimators.append(('feature_union', feature_union))
    #sv = SVC(kernel='rbf', gamma=0.1, probability=True)
    sv = SVC(kernel='linear', gamma=0.1, probability=True)
    tag = "SVC"
    useRF = True
    if useRF:
        sv = RandomForestClassifier()
        tag = "RF"
        #sv = linear_model.LogisticRegression()
        #tag = "logistic"
        
    estimators.append((tag, sv))
    model = Pipeline(estimators)
    # evaluate pipeline
    #num_folds = 10
    #num_instances = len(X)
    #seed = 7
    #kfold = KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    #results = cross_val_score(model, X, Y, cv=kfold)
    
        #sv = RandomForestClassifier()
    #param_sv = {"C": [1e0, 1e1, 1e2, 1e3],
    #                    "gamma": np.logspace(-2, 2, 5)}
    param_sv = {"SVC__C": [1e0, 1e1, 1e2, 1e3],
                        "SVC__gamma": np.logspace(-2, 2, 5)}
    
    param_lr = {
        #"logistic__n_components": [20, 40, 64],
                    "logistic__C" : np.logspace(-4, 4, 3)}
    param_rf = {"RF__max_depth": [3, 10],
              "RF__max_features": [1, 3],
              "RF__min_samples_split": [1, 3],
              "RF__min_samples_leaf": [1, 3],
              "RF__bootstrap": [True, False],
              #"criterion": ["gini", "entropy"]
              "RF__criterion": ["entropy"]
               }
    #tcv = TimeSeriesSplit(n_splits=4)
    cv_splits = 2
    svm = GridSearchCV(model, cv=cv_splits,
                       param_grid=param_sv)
    if useRF:
        svm = GridSearchCV(model, cv=cv_splits,
                       param_grid=param_rf)
        
    #print(x1)
    #print(y1)
    #print(x2)
    #print(y2)
    #Tracer()()
    #print("mean", x1.mean(axis=0), "std", x1.std(axis=0))

    svm.fit(x1, y1)
    #y1_svr = svm.predict(x1)
    #y2_svr = svm.predict(x2)
    #svm_num = svm.best_estimator_.named_steps["SVC"].n_support_
    #if useRF:
    #    svm_num = np.array([0])
    s_in = svm.score(x1, y1)
    s_out = svm.score(x2, y2)
    #y3 = np.full(len(y2), -1)
    #always do nothing
    #res_e_simple = accuracy_score(y2, y3)
    #cm = confusion_matrix(y2, y2_svr, labels=[-1, 1])
    #cm_simple = confusion_matrix(y3, y2_svr, labels=[-1, 1])
    
    le_in = log_loss(y1, svm.best_estimator_.named_steps[tag].predict_proba(x1))
    le_out = log_loss(y2, svm.best_estimator_.named_steps[tag].predict_proba(x2))
    
    print(k, le_in, le_out, s_in, s_out)
    #(log_loss(y[:train_size], gp_fix.predict_proba(X[:train_size])[:, 1])
     
    #print(mycm(y2, y2_svr))
    #print(cm)
    
    if do_plot:
        plt.figure()
        plt.clf()
        d = {"-1": "blue", "1":"yellow"}
        c1 = list(map(lambda x : "yellow" if x == -1 else "green", y1))
        patch1 = mpatches.Patch(color='yellow', label='-1 CNG<=0')
        patch2 = mpatches.Patch(color='green', label='+1 CNG>0')
        plt.legend(handles=[patch1, patch2])
        #print(x1[:, 0])
        #print(c1)
        plt.scatter(x1[:, 0], x1[:, 1], c=c1, zorder=10, cmap=plt.cm.Paired)
        #plt.scatter(x2[:, 0], x2[:, 1], s=80, facecolors='none', zorder=10)
        plt.scatter(x2[:, 0], x2[:, 1], s=280, c="k", marker='+', zorder=10, lw=2)

        #plt.axis('tight')
        x_min = x1[:, 0].min()
        x_max = x1[:, 0].max()
        y_min = x1[:, 1].min()
        y_max = x1[:, 1].max()
        print(svm.best_estimator_.named_steps["SVC"])
        XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]
        #Z = svm.decision_function(np.c_[XX.ravel(), YY.ravel()])
        Z = svm.best_estimator_.named_steps["SVC"].decision_function(np.c_[XX.ravel(), YY.ravel()])

        # Put the result into a color plot
        Z = Z.reshape(XX.shape)
        plt.pcolormesh(XX, YY, Z > 0, cmap=plt.cm.Paired)
        plt.contour(XX, YY, Z, colors=['k', 'k', 'k'], linestyles=['--', '-', '--'],
                levels=[-.5, 0, .5])

        plt.title("svm")#plt.plot(unprep(x1), y1_svr, 'o')
        plt.show()
        #plt.figure()
        #plt.plot(unprep(x2), y2_svr, 'o')
        #plt.figure()
        #plt.plot(res)
        #plt.figure()
        #plt.plot(unprep(x1), label ="X")
        #plt.plot(y1_svr, label ="y_in")
        #plt.legend(loc="best")
        #plt.figure()
        #plt.plot(unprep(x2), label ="X")
        #plt.plot(y2_svr, label ="y_out")
        #plt.legend(loc="best")
    return le_in, le_out, s_in, s_out

In [ ]:
#for i in range(2, df.shape[1]):
#    print(i)
le_in, le_out, s_in, s_out = sub_p_svm(x_train,y_train,x_test, y_test, 5, do_plot=False)

#logloss
#target values
#0.48

#plot_confusion_matrix(cm, ["-1", "1"])

#select best features



In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,
                          axis=0):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    orig = cm
    if normalize:
        if axis == 1:
            cm = cm.astype('float') / cm.sum(axis=axis)[:, np.newaxis]
        else:
            cm = cm.astype('float') / cm.sum(axis=axis)
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        #thresh = cm[i].max() / 2.
        #thresh = orig.sum(axis=axis)[i]*0.5
        thresh = orig.sum()*0.5
        #print(i, j, thresh, orig[i,j])
        #col = "black"
        s = "{:1.4f}".format(cm[i,j]) + "("+str(orig[i,j])+")"
        plt.text(j, i, s,
                 horizontalalignment="center",
                 color="white" if orig[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')